In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from tqdm import tqdm
import ast

from multiprocessing import Pool

In [2]:
secuencias_entrenamiento_df = pd.read_csv('../Data/secuencias_entrenamiento.csv')
secuencias_entrenamiento_df

,session_id,item_id_salida,secuencia_items_ids,secuencia_fechas
0,3,15085,"[9655, 9655]","['2020-12-18 21:19:48.093', '2020-12-18 21:25:..."
1,13,18626,[15654],['2020-03-13 19:35:27.136']
2,18,24911,"[4026, 2507, 18316]","['2020-08-26 19:15:47.232', '2020-08-26 19:16:..."
3,19,12534,"[19896, 27937, 12804, 25772, 6341, 23687, 8316...","['2020-11-02 16:30:36.378', '2020-11-02 16:30:..."
4,24,13226,"[2927, 2927, 16064, 11662, 434, 18539, 10414, ...","['2020-02-26 17:22:48.903', '2020-02-26 17:23:..."
...,...,...,...,...
999995,4439986,2915,"[23502, 13914, 13403, 19310, 12373, 27733]","['2021-05-13 11:46:56.606', '2021-05-13 11:48:..."
999996,4439990,8786,"[22093, 17429, 1389, 1131, 4062, 26011, 4028, ...","['2020-08-22 11:38:48.785', '2020-08-22 11:39:..."
999997,4439994,21630,[25357],['2020-11-27 20:08:37.262']
999998,4439999,16962,"[6007, 8502, 13056, 2173, 8433, 1299, 15853]","['2020-11-27 10:52:12.577', '2020-11-27 10:53:..."


In [3]:
encoded_items_scaled_menos_uno_a_uno_df = pd.read_csv('../Data/items_autoencoder_encoded_scaled_menos_uno_a_uno.csv')
encoded_items_scaled_menos_uno_a_uno_df

,item_id,0,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,2,-0.365605,0.118462,-0.010897,-0.320781,0.257142,0.537237,-0.188795,0.189533,-0.156756,...,-0.143641,0.325997,0.298440,-0.017123,0.063813,0.261511,0.278820,-0.004606,0.541165,0.278077
1,3,-0.174989,-0.029986,-0.442100,0.028794,-0.487501,-0.178628,-0.281027,0.495853,0.031953,...,-0.013423,0.341105,0.055358,0.285594,-0.215148,-0.398990,0.426400,-0.014106,0.052011,-0.446843
2,4,-0.366774,-0.522646,0.095210,-0.147176,-0.097715,-0.396698,0.044612,0.337221,-0.049645,...,-0.096266,0.378600,0.096531,0.340519,0.179593,-0.114074,-0.300024,-0.091382,0.201742,0.239278
3,7,-0.150294,0.214653,-0.256845,0.180945,0.280534,0.571016,-0.107931,0.256094,0.243286,...,0.228527,0.253088,-0.196190,0.085781,0.261103,-0.421948,0.208436,-0.043876,0.282139,0.077041
4,8,-0.405001,-0.161389,-0.137880,-0.132782,-0.445178,-0.204801,-0.670361,0.433809,0.185705,...,-0.461848,0.097494,-0.313996,0.208256,-0.303381,0.079611,-0.089671,-0.241595,0.278577,-0.354867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,-0.446965,-0.076762,0.258293,-0.134691,-0.617824,-0.350357,-0.056547,-0.194005,-0.026162,...,-0.184961,-0.281994,-0.197465,0.331412,-0.011554,0.225237,-0.134212,-0.005801,-0.111278,-0.774558
23687,28140,0.104798,0.170240,-0.098097,-0.202364,0.135956,-0.110908,0.044891,0.456119,0.338910,...,-0.100782,0.456120,-0.259156,-0.275627,-0.622236,-0.314829,0.295296,-0.440746,-0.253330,-0.139412
23688,28141,0.168311,-0.137270,-0.078370,0.092640,0.024696,-0.379267,-0.040895,0.380580,-0.126175,...,0.266188,0.356827,-0.084128,0.154201,-0.344023,-0.285257,0.141432,-0.085466,0.137665,-0.298925
23689,28142,-0.245331,-0.133603,-0.095151,-0.195275,0.019389,0.133851,-0.420393,-0.039917,0.122732,...,-0.022007,-0.032347,0.679219,-0.110100,0.199421,-0.197126,0.443335,0.037485,0.288340,0.282589


In [198]:
def normalize_len_sequence(sequence_list, len_output = 10):
    
    if len(sequence_list) >= len_output:
        
        output_list = sequence_list[-1*len_output:]
        
    else:
        
        diference_len = len_output - len(sequence_list)
        padding = [0] * diference_len 
        output_list = padding + sequence_list
        
    return output_list


def create_tensors_for_rnn(secuencias_df, items_df, length_sequence = 10):
    
    X = []
    Y = []
    
    example_tensor = items_df.loc[[0]].drop('item_id', axis=1).to_numpy()
    
    for index, row in tqdm(secuencias_df.iterrows()):
        
        secuencia_items = ast.literal_eval(row.secuencia_items_ids)
        secuencia_items_normalizada = normalize_len_sequence(secuencia_items, length_sequence)
        
        secuencia_fechas_items = ast.literal_eval(row.secuencia_fechas)
        secuencia_fechas_items_normalizada = normalize_len_sequence(secuencia_fechas_items, length_sequence)
        
        X_secuencia_individual = []
        
        for item in secuencia_items_normalizada:
                        
            if item == 0:
                
                item_data_tensor = np.zeros(example_tensor.shape)
                X_secuencia_individual.append(item_data_tensor)
                
            else:
                
                item_data_tensor = items_df.loc[items_df.item_id == item].drop('item_id', axis=1).to_numpy()
                X_secuencia_individual.append(item_data_tensor)
        
        
        X_secuencia_individual = np.array(X_secuencia_individual)
        X_secuencia_individual = np.reshape(X_secuencia_individual, (X_secuencia_individual.shape[0], X_secuencia_individual.shape[2]))
        
        X.append(X_secuencia_individual)
        Y.append(items_df.loc[items_df.item_id == row.item_id_salida].drop('item_id', axis=1).to_numpy())
        

    X = np.array(X)
    Y = np.array(Y)
    Y = np.reshape(Y, (Y.shape[0], Y.shape[2]))
    
    return X, Y

# Creacion de un dataset con secuencias de una longitud de 10 sesiones con los valores de los items en espacio latente del autoencoder

In [199]:
X_10,Y_10 = create_tensors_for_rnn(secuencias_entrenamiento_df, encoded_items_scaled_menos_uno_a_uno_df, 10)

1000000it [29:41, 561.27it/s]


In [200]:
X_10.shape

(1000000, 10, 101)

In [201]:
Y_10.shape

(1000000, 101)

In [202]:
np.save('../Data/X_10', X_10)
np.save('../Data/Y_10', Y_10)